# Real-Time Trading Simulation with Chan System

This notebook:
1. Processes raw SPY_5m.csv data through the Chan system
2. Detects BSPoints as they mature in real-time
3. Trains models dynamically on rolling windows
4. Executes trades based on model predictions
5. Tracks performance vs. Buy & Hold

## Key Features
- ✅ True sequential processing (no look-ahead bias)
- ✅ BSPoints generated on-the-fly
- ✅ Dynamic model training with rolling windows
- ✅ Realistic trade execution timing
- ✅ Comprehensive performance tracking

## 1. Setup and Imports

In [ ]:
# Core imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from collections import deque, defaultdict
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Get current directory and add parent directory to path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
print(f"📁 Current directory: {current_dir}")
print(f"📁 Parent directory: {parent_dir}")

# Add parent directory to path (where Chan system files are)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"✅ Added parent directory to Python path")

# Also add current directory
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Import Chan system components
try:
    from sliding_window_chan import SlidingWindowChan
    from ChanConfig import CChanConfig
    from Common.CEnum import KL_TYPE, DATA_SRC, AUTYPE
    print("✅ Chan system imports successful!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print(f"\n🔍 Please ensure these files are in parent directory:")
    print(f"  - sliding_window_chan.py")
    print(f"  - ChanConfig.py")
    print(f"  - Common/CEnum.py")
    print(f"\n📂 Current sys.path:")
    for p in sys.path[:5]:
        print(f"  - {p}")
    raise

# Setup recursion limit
sys.setrecursionlimit(30000)

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"\n✅ All imports successful!")
print(f"🐍 Python version: {sys.version.split()[0]}")

## 2. Configuration

In [ ]:
# ====================
# DATA CONFIGURATION
# ====================
# Adjust path relative to train folder
DATA_PATH = "../backtestData/SPY_5m.csv"  # Parent directory's backtestData folder
OUTPUT_DIR = "./output/realtime_chan_simulation"

# Verify data file exists
if os.path.exists(DATA_PATH):
    print(f"✅ Found data file: {DATA_PATH}")
else:
    print(f"❌ Data file not found: {DATA_PATH}")
    print(f"\n🔍 Looking for data file in:")
    # Try different locations
    possible_paths = [
        "../backtestData/SPY_5m.csv",
        "./backtestData/SPY_5m.csv",
        "../../backtestData/SPY_5m.csv",
        "./SPY_5m.csv",
    ]
    for path in possible_paths:
        if os.path.exists(path):
            DATA_PATH = path
            print(f"✅ Found data at: {DATA_PATH}")
            break
        else:
            print(f"  ❌ Not found: {path}")
    else:
        print(f"\n⚠️ Please update DATA_PATH to point to your SPY_5m.csv file")

# ====================
# CHAN CONFIGURATION
# ====================
TICKER = "SPY"
BEGIN_TIME = "2021-01-01"  # Start date for processing
END_TIME = "2025-01-01"    # End date for processing
LEVEL = KL_TYPE.K_5M       # K-line level
CHAN_WINDOW_SIZE = 500     # K-lines per Chan calculation window

# Chan system configuration
chan_config = CChanConfig({
    "cal_demark": True,
    "cal_kdj": True,
    "cal_dmi": True,
    "cal_rsi": True,
    "cal_macd": True,
    "bi_strict": True,
    "trigger_step": False,  # Not needed for sliding window
    "bs_type": '1,2,3a,1p,2s,3b',
    "print_warning": False,
})

# ====================
# TRADING CONFIGURATION
# ====================
TRAINING_WINDOW_DAYS = 30      # Days of history for training
MIN_TRAINING_SAMPLES = 50      # Minimum samples needed per direction
RETRAIN_FREQUENCY_DAYS = 7     # Retrain every N days
POSITION_SIZE = 1.0            # Fraction of capital per trade (1.0 = 100%)
INITIAL_CAPITAL = 10000        # Starting capital
TRANSACTION_COST = 0.001       # 0.1% transaction cost

# ====================
# MODEL CONFIGURATION
# ====================
MODEL_PARAMS = {
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'random_state': 42,
    'verbosity': 0
}

# ====================
# PROFIT TARGET CONFIGURATION
# ====================
TIME_WINDOW_BARS = 288  # 24 hours = 288 5-min bars

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("="*80)
print("CONFIGURATION SUMMARY")
print("="*80)
print(f"📊 Ticker: {TICKER}")
print(f"📅 Period: {BEGIN_TIME} to {END_TIME}")
print(f"📈 Level: {LEVEL.name}")
print(f"🪟 Chan Window: {CHAN_WINDOW_SIZE} K-lines")
print(f"💰 Initial Capital: ${INITIAL_CAPITAL:,.2f}")
print(f"📊 Position Size: {POSITION_SIZE*100}%")
print(f"🎓 Training Window: {TRAINING_WINDOW_DAYS} days")
print(f"🔄 Retrain Frequency: {RETRAIN_FREQUENCY_DAYS} days")
print(f"📁 Output: {OUTPUT_DIR}")
print(f"📂 Data: {DATA_PATH}")
print("="*80)

## 3. Helper Functions

In [ ]:
def calculate_profit_target_best_within_24h(bs_points_list: List[Dict], time_window_bars: int = 288) -> Dict:
    """
    Calculate profit targets using BEST reverse signal within time window.
    For BUY: Find SELL with HIGHEST price (max profit)
    For SELL: Find BUY with LOWEST price (max profit)
    """
    # Sort by klu_idx
    bs_points_list.sort(key=lambda x: x['klu_idx'])
    
    profit_targets = {}
    
    for i, data in enumerate(bs_points_list):
        current_direction = data['is_buy']
        entry_price = data['klu_close']
        entry_idx = data['klu_idx']
        
        # Initialize
        profit_targets[entry_idx] = {
            'profit_target_pct': None,
            'has_profit_target': 0,
            'exit_klu_idx': None,
            'exit_price': None,
        }
        
        # Define search window
        window_end_idx = entry_idx + time_window_bars
        
        # Find reverse signals within window
        candidates = []
        fallback = None
        
        for j in range(i + 1, len(bs_points_list)):
            future_data = bs_points_list[j]
            future_idx = future_data['klu_idx']
            
            if future_data['is_buy'] != current_direction:
                if fallback is None:
                    fallback = future_data
                
                if future_idx <= window_end_idx:
                    candidates.append(future_data)
        
        # Select best candidate
        best_exit = None
        if candidates:
            if current_direction:  # BUY -> find SELL with highest price
                best_exit = max(candidates, key=lambda x: x['klu_close'])
            else:  # SELL -> find BUY with lowest price
                best_exit = min(candidates, key=lambda x: x['klu_close'])
        elif fallback:
            best_exit = fallback
        
        if best_exit:
            exit_price = best_exit['klu_close']
            
            if current_direction:  # BUY -> SELL
                profit_pct = (exit_price - entry_price) / entry_price * 100
            else:  # SELL -> BUY
                profit_pct = (entry_price - exit_price) / entry_price * 100
            
            profit_targets[entry_idx] = {
                'profit_target_pct': profit_pct,
                'has_profit_target': 1,
                'exit_klu_idx': best_exit['klu_idx'],
                'exit_price': exit_price,
            }
    
    return profit_targets


def extract_features_from_bsp(bsp_dict: Dict) -> Optional[Dict]:
    """
    Extract feature vector from BSPoint dictionary.
    Returns None if BSPoint doesn't have required features.
    """
    if not bsp_dict.get('has_profit_target', 0):
        return None
    
    features = {}
    
    # Exclude metadata columns
    exclude_patterns = [
        'timestamp', 'bsp_type', 'direction', 'profit_target', 
        'has_profit', 'return_', 'label_', 'target_return_',
        'snapshot_', 'klu_idx', 'exit_', 'date', 'is_buy',
        'klu_open', 'klu_high', 'klu_low', 'klu_close', 'bsp_types'
    ]
    
    for key, value in bsp_dict.items():
        if any(pat in key for pat in exclude_patterns):
            continue
        
        if not isinstance(value, (int, float)):
            continue
        
        features[key] = value if pd.notna(value) else 0.0
    
    return features if features else None


print("✅ Helper functions defined")

## 4. Real-Time Trading Simulator Class

In [ ]:
class ChanRealtimeSimulator:
    """
    Real-time trading simulator that processes data through Chan system.
    """
    
    def __init__(
        self,
        ticker: str,
        begin_time: str,
        end_time: str,
        data_src: DATA_SRC,
        level: KL_TYPE,
        config: CChanConfig,
        output_dir: str,
        training_window_days: int = 30,
        min_training_samples: int = 50,
        retrain_frequency_days: int = 7,
        position_size: float = 1.0,
        initial_capital: float = 10000,
        transaction_cost: float = 0.001,
        model_params: Optional[Dict] = None,
        chan_window_size: int = 500,
        time_window_bars: int = 288,
    ):
        self.ticker = ticker
        self.begin_time = begin_time
        self.end_time = end_time
        self.data_src = data_src
        self.level = level
        self.config = config
        self.output_dir = Path(output_dir)
        
        # Training parameters
        self.training_window_days = training_window_days
        self.min_training_samples = min_training_samples
        self.retrain_frequency_days = retrain_frequency_days
        
        # Trading parameters
        self.position_size = position_size
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        
        # Model parameters
        self.model_params = model_params or MODEL_PARAMS
        
        # Chan parameters
        self.chan_window_size = chan_window_size
        self.time_window_bars = time_window_bars
        
        # State tracking
        self.all_bsp_history: List[Dict] = []  # All BSPs (mature + immature)
        self.mature_bsp_history: List[Dict] = []  # Only mature BSPs
        self.processed_bsp_keys: set = set()  # Track processed BSPs
        
        # Models
        self.buy_model: Optional[xgb.XGBRegressor] = None
        self.sell_model: Optional[xgb.XGBRegressor] = None
        self.last_training_date: Optional[datetime] = None
        self.feature_cols: Optional[List[str]] = None
        
        # Trading state
        self.cash = initial_capital
        self.shares = 0
        self.entry_price = 0
        self.trades: List[Dict] = []
        self.equity_curve: List[Dict] = []
        
        print("✅ ChanRealtimeSimulator initialized")
    
    def get_training_data(self, current_date: datetime, direction: str) -> Tuple[pd.DataFrame, pd.Series]:
        """Get training data from mature BSPoints within training window."""
        training_start = current_date - timedelta(days=self.training_window_days)
        
        # Filter BSPoints
        training_bsps = [
            bsp for bsp in self.mature_bsp_history
            if (training_start <= bsp['timestamp'] <= current_date and
                bsp['direction'] == direction)
        ]
        
        if len(training_bsps) < self.min_training_samples:
            return None, None
        
        # Extract features
        feature_dicts = []
        targets = []
        
        for bsp in training_bsps:
            features = extract_features_from_bsp(bsp)
            if features:
                feature_dicts.append(features)
                targets.append(bsp['profit_target_pct'])
        
        if not feature_dicts:
            return None, None
        
        # Convert to DataFrame
        X = pd.DataFrame(feature_dicts)
        y = pd.Series(targets)
        
        # Ensure consistent features
        if self.feature_cols is None:
            self.feature_cols = sorted(X.columns)
        else:
            for col in self.feature_cols:
                if col not in X.columns:
                    X[col] = 0
            X = X[self.feature_cols]
        
        return X, y
    
    def train_models(self, current_date: datetime) -> bool:
        """Train buy and sell models."""
        # Get training data
        X_buy, y_buy = self.get_training_data(current_date, 'buy')
        X_sell, y_sell = self.get_training_data(current_date, 'sell')
        
        if X_buy is None or X_sell is None:
            return False
        
        # Train models
        self.buy_model = xgb.XGBRegressor(**self.model_params)
        self.buy_model.fit(X_buy, y_buy)
        
        self.sell_model = xgb.XGBRegressor(**self.model_params)
        self.sell_model.fit(X_sell, y_sell)
        
        self.last_training_date = current_date
        
        print(f"  [Training] ✓ Models trained: Buy={len(X_buy)} samples, Sell={len(X_sell)} samples")
        return True
    
    def should_retrain(self, current_date: datetime) -> bool:
        """Check if models should be retrained."""
        if self.last_training_date is None:
            return True
        days_since = (current_date - self.last_training_date).days
        return days_since >= self.retrain_frequency_days
    
    def predict_profit(self, bsp_dict: Dict) -> Optional[float]:
        """Predict profit for a BSPoint."""
        if self.buy_model is None or self.sell_model is None:
            return None
        
        features = extract_features_from_bsp(bsp_dict)
        if not features:
            return None
        
        X = pd.DataFrame([features])
        for col in self.feature_cols:
            if col not in X.columns:
                X[col] = 0
        X = X[self.feature_cols]
        
        direction = bsp_dict['direction']
        if direction == 'buy':
            return self.buy_model.predict(X)[0]
        else:
            return self.sell_model.predict(X)[0]
    
    def execute_trade(self, bsp_dict: Dict, predicted_profit: float):
        """Execute a trade."""
        current_price = bsp_dict['klu_close']
        timestamp = bsp_dict['timestamp']
        direction = bsp_dict['direction']
        
        if direction == 'buy' and self.shares == 0:
            trade_capital = self.cash * self.position_size
            self.shares = trade_capital / current_price
            cost = self.shares * current_price * (1 + self.transaction_cost)
            self.cash -= cost
            self.entry_price = current_price
            
            self.trades.append({
                'timestamp': timestamp,
                'type': 'BUY',
                'price': current_price,
                'shares': self.shares,
                'predicted_profit': predicted_profit,
                'actual_profit': None
            })
        
        elif direction == 'sell' and self.shares > 0:
            exit_value = self.shares * current_price * (1 - self.transaction_cost)
            trade_profit = exit_value - (self.shares * self.entry_price)
            trade_return_pct = (trade_profit / (self.shares * self.entry_price)) * 100
            
            self.cash += exit_value
            
            if self.trades:
                self.trades[-1]['actual_profit'] = trade_return_pct
            
            self.shares = 0
            self.entry_price = 0
    
    def update_equity_curve(self, timestamp: datetime, current_price: float):
        """Update equity curve."""
        if self.shares > 0:
            portfolio_value = self.cash + (self.shares * current_price)
        else:
            portfolio_value = self.cash
        
        self.equity_curve.append({
            'timestamp': timestamp,
            'portfolio_value': portfolio_value,
            'cash': self.cash,
            'shares': self.shares,
            'current_price': current_price
        })
    
    def run_simulation(self):
        """Main simulation loop."""
        print("\n" + "="*80)
        print("STARTING REAL-TIME CHAN SIMULATION")
        print("="*80)
        
        # Initialize Chan system
        print("\n[1/4] Initializing Chan system...")
        chan = SlidingWindowChan(
            code=self.ticker,
            begin_time=self.begin_time,
            end_time=self.end_time,
            data_src=self.data_src,
            lv_list=[self.level],
            config=self.config,
            autype=AUTYPE.QFQ,
            max_klines=self.chan_window_size
        )
        
        # Process snapshots
        print("\n[2/4] Processing K-lines and detecting BSPoints...")
        snapshot_count = 0
        last_print_time = datetime.now()
        
        for snapshot_idx, snapshot_chan in enumerate(chan.step_load()):
            snapshot_count += 1
            
            # Get current BSPoints
            current_bsps = chan.get_all_historical_bsp()
            
            # Calculate profit targets for new BSPs
            if len(current_bsps) > len(self.all_bsp_history):
                # Calculate profit targets for all BSPs
                profit_targets = calculate_profit_target_best_within_24h(
                    current_bsps,
                    self.time_window_bars
                )
                
                # Update BSP history
                self.all_bsp_history = current_bsps
                
                # Check for new mature BSPs
                for bsp in current_bsps:
                    klu_idx = bsp['klu_idx']
                    bsp_key = (str(bsp['timestamp']), bsp['bsp_type'])
                    
                    # Add profit target info
                    if klu_idx in profit_targets:
                        bsp.update(profit_targets[klu_idx])
                    
                    # Check if mature and not processed
                    if bsp.get('has_profit_target', 0) and bsp_key not in self.processed_bsp_keys:
                        self.processed_bsp_keys.add(bsp_key)
                        self.mature_bsp_history.append(bsp)
                        
                        current_date = bsp['timestamp']
                        
                        # Train models if needed
                        if self.should_retrain(current_date):
                            self.train_models(current_date)
                        
                        # Make trading decision
                        if self.buy_model is not None and self.sell_model is not None:
                            predicted_profit = self.predict_profit(bsp)
                            
                            if predicted_profit is not None and predicted_profit > 0:
                                self.execute_trade(bsp, predicted_profit)
                        
                        # Update equity
                        self.update_equity_curve(current_date, bsp['klu_close'])
            
            # Progress reporting
            now = datetime.now()
            if snapshot_count % 500 == 0 or (now - last_print_time).total_seconds() > 10:
                mature_count = len(self.mature_bsp_history)
                print(f"  Snapshot {snapshot_count:,} | Mature BSPs: {mature_count} | Trades: {len(self.trades)}")
                last_print_time = now
        
        # Close open position
        if self.shares > 0 and self.mature_bsp_history:
            final_bsp = self.mature_bsp_history[-1]
            final_price = final_bsp['klu_close']
            exit_value = self.shares * final_price
            self.cash += exit_value
            print(f"\n  [TRADE] Closing position: SELL {self.shares:.2f} shares @ ${final_price:.2f}")
            self.shares = 0
        
        print(f"\n[3/4] Simulation complete!")
        print(f"  Total snapshots: {snapshot_count:,}")
        print(f"  Mature BSPoints: {len(self.mature_bsp_history)}")
        print(f"  Total trades: {len(self.trades)}")
    
    def analyze_results(self):
        """Analyze and visualize results."""
        print("\n[4/4] Analyzing results...")
        
        if not self.equity_curve:
            print("  No equity data to analyze")
            return None, None
        
        # Convert to DataFrames
        equity_df = pd.DataFrame(self.equity_curve)
        trades_df = pd.DataFrame(self.trades) if self.trades else pd.DataFrame()
        
        # Calculate returns
        final_value = equity_df['portfolio_value'].iloc[-1]
        total_return = ((final_value / self.initial_capital) - 1) * 100
        
        # Buy & Hold
        first_price = equity_df['current_price'].iloc[0]
        last_price = equity_df['current_price'].iloc[-1]
        bh_return = ((last_price / first_price) - 1) * 100
        
        # Trade statistics
        completed_trades = trades_df[trades_df['actual_profit'].notna()] if len(trades_df) > 0 else pd.DataFrame()
        
        if len(completed_trades) > 0:
            win_rate = (completed_trades['actual_profit'] > 0).mean() * 100
            avg_profit = completed_trades['actual_profit'].mean()
        else:
            win_rate = avg_profit = 0
        
        # Print summary
        print("\n" + "="*80)
        print("RESULTS SUMMARY")
        print("="*80)
        print(f"Initial capital: ${self.initial_capital:,.2f}")
        print(f"Final value: ${final_value:,.2f}")
        print(f"Total return: {total_return:+.2f}%")
        print(f"Buy & Hold return: {bh_return:+.2f}%")
        print(f"Outperformance: {total_return - bh_return:+.2f}%")
        print(f"\nTrading statistics:")
        print(f"  Total trades: {len(trades_df)}")
        print(f"  Completed trades: {len(completed_trades)}")
        print(f"  Win rate: {win_rate:.1f}%")
        print(f"  Average profit: {avg_profit:.2f}%")
        
        # Save results
        equity_df.to_csv(self.output_dir / 'equity_curve.csv', index=False)
        if len(trades_df) > 0:
            trades_df.to_csv(self.output_dir / 'trades.csv', index=False)
        if self.mature_bsp_history:
            pd.DataFrame(self.mature_bsp_history).to_csv(self.output_dir / 'mature_bspoints.csv', index=False)
        
        print(f"\n✅ Results saved to: {self.output_dir}/")
        
        return equity_df, trades_df


print("✅ ChanRealtimeSimulator class defined")

## 5. Run the Simulation

In [ ]:
# Create simulator
simulator = ChanRealtimeSimulator(
    ticker=TICKER,
    begin_time=BEGIN_TIME,
    end_time=END_TIME,
    data_src=DATA_SRC.CSV,
    level=LEVEL,
    config=chan_config,
    output_dir=OUTPUT_DIR,
    training_window_days=TRAINING_WINDOW_DAYS,
    min_training_samples=MIN_TRAINING_SAMPLES,
    retrain_frequency_days=RETRAIN_FREQUENCY_DAYS,
    position_size=POSITION_SIZE,
    initial_capital=INITIAL_CAPITAL,
    transaction_cost=TRANSACTION_COST,
    model_params=MODEL_PARAMS,
    chan_window_size=CHAN_WINDOW_SIZE,
    time_window_bars=TIME_WINDOW_BARS,
)

# Run simulation
simulator.run_simulation()

# Analyze results
equity_df, trades_df = simulator.analyze_results()

## 6. Visualize Results

In [ ]:
# Only create visualizations if we have data
if equity_df is not None and len(equity_df) > 0:
    # Create visualizations
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))

    # 1. Equity curve
    ax1 = axes[0]
    ax1.plot(equity_df['timestamp'], equity_df['portfolio_value'], label='Strategy', linewidth=2)

    # Buy & Hold
    first_price = equity_df['current_price'].iloc[0]
    bh_value = INITIAL_CAPITAL * (equity_df['current_price'] / first_price)
    ax1.plot(equity_df['timestamp'], bh_value, label='Buy & Hold', linewidth=2, alpha=0.7)

    ax1.axhline(y=INITIAL_CAPITAL, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
    ax1.set_title('Equity Curve: Real-Time Simulation vs. Buy & Hold', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.legend(loc='best')
    ax1.grid(alpha=0.3)

    # 2. Cumulative returns
    ax2 = axes[1]
    strategy_returns = ((equity_df['portfolio_value'] / INITIAL_CAPITAL) - 1) * 100
    bh_returns = ((bh_value / INITIAL_CAPITAL) - 1) * 100

    ax2.plot(equity_df['timestamp'], strategy_returns, label='Strategy', linewidth=2)
    ax2.plot(equity_df['timestamp'], bh_returns, label='Buy & Hold', linewidth=2, alpha=0.7)
    ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax2.set_title('Cumulative Returns (%)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Return (%)')
    ax2.legend(loc='best')
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/equity_curve.png", dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\n✅ Charts saved to {OUTPUT_DIR}/equity_curve.png")
else:
    print("\n⚠️ No data to visualize")

## 7. Trade Analysis

In [ ]:
if trades_df is not None and len(trades_df) > 0:
    completed = trades_df[trades_df['actual_profit'].notna()]
    
    if len(completed) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Trade distribution
        ax1 = axes[0]
        ax1.hist(completed['actual_profit'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Breakeven')
        ax1.set_xlabel('Trade Return (%)', fontsize=12)
        ax1.set_ylabel('Frequency', fontsize=12)
        ax1.set_title('Distribution of Trade Returns', fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(alpha=0.3)
        
        # Predicted vs Actual
        ax2 = axes[1]
        ax2.scatter(completed['predicted_profit'], completed['actual_profit'], alpha=0.6, s=50)
        ax2.plot([-5, 5], [-5, 5], 'r--', linewidth=2, label='Perfect prediction')
        ax2.set_xlabel('Predicted Profit (%)', fontsize=12)
        ax2.set_ylabel('Actual Profit (%)', fontsize=12)
        ax2.set_title('Predicted vs. Actual Profit', fontsize=14, fontweight='bold')
        ax2.legend()
        ax2.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/trade_analysis.png", dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\n✅ Trade analysis saved to {OUTPUT_DIR}/trade_analysis.png")
        
        # Display trade summary
        print("\n" + "="*80)
        print("TRADE SUMMARY")
        print("="*80)
        print(completed[['timestamp', 'type', 'price', 'predicted_profit', 'actual_profit']].tail(10))
else:
    print("\n⚠️ No completed trades to analyze")

## 8. BSPoint Analysis

In [ ]:
if simulator.mature_bsp_history:
    bsp_df = pd.DataFrame(simulator.mature_bsp_history)
    
    print("\n" + "="*80)
    print("BSPOINT ANALYSIS")
    print("="*80)
    
    print(f"\nTotal mature BSPoints: {len(bsp_df)}")
    
    # Direction distribution
    print("\nDirection Distribution:")
    direction_counts = bsp_df['direction'].value_counts()
    for direction, count in direction_counts.items():
        pct = count / len(bsp_df) * 100
        print(f"  {direction.capitalize()}: {count} ({pct:.1f}%)")
    
    # BSP type distribution
    if 'bsp_type' in bsp_df.columns:
        print("\nBSP Type Distribution:")
        type_counts = bsp_df['bsp_type'].value_counts()
        for bsp_type, count in type_counts.items():
            pct = count / len(bsp_df) * 100
            print(f"  {bsp_type}: {count} ({pct:.1f}%)")
    
    # Profit statistics
    if 'profit_target_pct' in bsp_df.columns:
        profit_data = bsp_df['profit_target_pct'].dropna()
        if len(profit_data) > 0:
            print("\nProfit Target Statistics:")
            print(f"  Mean: {profit_data.mean():.2f}%")
            print(f"  Median: {profit_data.median():.2f}%")
            print(f"  Std Dev: {profit_data.std():.2f}%")
            print(f"  Min: {profit_data.min():.2f}%")
            print(f"  Max: {profit_data.max():.2f}%")
            print(f"  Profitable: {(profit_data > 0).sum()} ({(profit_data > 0).mean()*100:.1f}%)")
    
    # Sample BSPoints
    print("\nSample BSPoints:")
    sample_cols = ['timestamp', 'bsp_type', 'direction', 'klu_close', 'profit_target_pct']
    available_cols = [col for col in sample_cols if col in bsp_df.columns]
    print(bsp_df[available_cols].head(10).to_string())
else:
    print("\n⚠️ No mature BSPoints to analyze")

## 9. Summary and Next Steps

In [ ]:
print("\n" + "="*80)
print("SIMULATION COMPLETE")
print("="*80)

print("\n📁 Output Files:")
for file in sorted(Path(OUTPUT_DIR).glob('*')):
    print(f"  ✓ {file.name}")

print("\n🎯 What This Simulation Achieved:")
print("  ✅ Processed raw data through Chan system sequentially")
print("  ✅ Detected BSPoints as they matured in real-time")
print("  ✅ Trained models dynamically on rolling windows")
print("  ✅ Made trading decisions without look-ahead bias")
print("  ✅ Tracked performance vs. Buy & Hold")

print("\n🚀 Next Steps:")
print("  1. Review equity_curve.png for performance visualization")
print("  2. Analyze trades.csv for detailed trade history")
print("  3. Examine mature_bspoints.csv for signal quality")
print("  4. Adjust parameters in Cell 2 and re-run")
print("  5. Compare with offline backtesting results")

print("\n💡 Configuration Tips:")
print("  - Increase training_window_days for more stable models")
print("  - Decrease retrain_frequency_days for more adaptive models")
print("  - Adjust position_size to manage risk")
print("  - Modify chan_window_size for speed vs. context tradeoff")

print("\n" + "="*80)